This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/lyapunov.html).  I recommend having both windows open, side-by-side!


In [ ]:
import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import Markdown, display
from pydrake.all import (
    BatchEvalTimeDerivatives,
    ExtractGradient,
    ExtractValue,
    InitializeAutoDiff,
    InputPortSelection,
    MathematicalProgram,
    Solve,
    ToLatex,
    Variable,
    Variables,
)
from pydrake.examples import PendulumParams, PendulumPlant

from underactuated import running_as_notebook

if running_as_notebook:
    mpld3.enable_notebook()

# Global stability of the simple pendulum via sampling + linear programming

In [ ]:
def basis_functions(x):
    one = 1 + 0 * x[0]
    s = np.sin(x[0])
    c = np.cos(x[0])
    qd = x[1]
    return np.vstack((one, s, c, qd, s * s, s * c, s * qd, c * qd, qd * qd))


def global_pendulum():
    pendulum = PendulumPlant()
    context = pendulum.CreateDefaultContext()
    pendulum.get_input_port().FixValue(context, 0.0)

    q_samples = np.linspace(-1.5 * np.pi, 1.5 * np.pi, 31)
    qd_samples = np.linspace(-10.0, 10.0, 21)
    x_samples = np.vstack(np.meshgrid(q_samples, qd_samples)).reshape(2, -1)
    num_samples = x_samples.shape[1]

    xdot_samples = BatchEvalTimeDerivatives(
        pendulum, context, [0] * num_samples, x_samples, [], InputPortSelection.kNoInput
    )

    x0 = np.zeros((2, 1))
    phi0 = basis_functions(x0)
    num_alpha = phi0.shape[0]

    prog = MathematicalProgram()

    slack = prog.NewContinuousVariables(num_samples, "s")
    alpha = prog.NewContinuousVariables(num_alpha, "a")

    # Minimize ∑ sᵢ
    prog.AddLinearCost(np.ones((num_samples, 1)), slack)

    # V(0) = 0
    prog.AddLinearEqualityConstraint(phi0.T, [0], alpha)

    for i in range(num_samples):
        x = x_samples[:, i]
        xdot = xdot_samples[:, i]
        phi_ad = basis_functions(InitializeAutoDiff(x))
        phi = ExtractValue(phi_ad)
        dphidx = ExtractGradient(phi_ad)
        phidot = dphidx @ xdot
        # V(xᵢ) ≥ 0
        prog.AddLinearConstraint(phi.T, 0, np.inf, alpha)

        # V̇(xᵢ) = ∂V/∂x f(xᵢ) ≤ 0.
        prog.AddLinearConstraint(phidot.T, -np.inf, 0, alpha)

        # sᵢ ≥ |V̇(xᵢ) + 1|.
        Vdot = alpha.dot(phidot)
        prog.AddLinearConstraint(slack[i] >= Vdot + 1)
        prog.AddLinearConstraint(slack[i] >= -(Vdot + 1))

    # Call the solver.
    result = Solve(prog)
    assert result.is_success()

    # Note that I've added mgl to the potential energy (relative to the textbook),
    # so that it would be non-negative... like the Lyapunov function.
    p = PendulumParams()
    mgl = p.mass() * p.gravity() * p.length()
    display(
        Markdown(f"$E = {0.5*p.mass() * p.length()**2} \dot\\theta^2 + {mgl}(1-c)$\n")
    )

    alpha_sol = result.GetSolution(alpha)

    x = np.array([Variable("\\theta"), Variable("\dot\\theta")])
    V_sol = alpha_sol.dot(basis_functions(x))
    display(Markdown(f"$V = {ToLatex(V_sol[0])}$\n"))

    # Plot the results as contour plots.
    nq = 151
    nqd = 151
    q = np.linspace(-2 * np.pi, 2 * np.pi, nq)
    qd = np.linspace(-2 * mgl, 2 * mgl, nqd)
    Q, QD = np.meshgrid(q, qd)
    Energy = 0.5 * p.mass() * p.length() ** 2 * QD**2 + mgl * (1 - np.cos(Q))
    Vplot = Q.copy()
    for i in range(nq):
        for j in range(nqd):
            Vplot[i, j] = alpha_sol.dot(basis_functions([Q[i, j], QD[i, j]]))[0]

    # plt.rc("text", usetex=True)
    fig, ax = plt.subplots()
    ax.contour(Q, QD, Vplot)
    ax.contour(Q, QD, Energy, alpha=0.5, linestyles="dashed")
    ax.set_xlabel("theta")
    ax.set_ylabel("thetadot")
    ax.set_title("V (solid) and Mechanical Energy (dashed)")


global_pendulum()